In [1]:
import numpy as np
from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import figure, show
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
import pickle
import imageio

In [2]:
output_notebook()

Loading BokehJS ...

In [19]:
CABLE_DELAYS = 0.65 * 2
DATA_DIRS = [dI for dI in os.listdir() if (os.path.isdir(dI) and not dI[0] == '.')]
DATA_DIRS.sort(reverse=True)
FOLDER_NAME = DATA_DIRS[0]

In [20]:
def pick_capture(capture):
    FOLDER_NAME = capture
    push_notebook()

In [21]:
with open(os.path.join(FOLDER_NAME, 'data.pickle'), 'rb') as f:
    frequencies = pickle.load(f)
    sweeps = pickle.load(f)
    frame_time = pickle.load(f)
sweeps = [tmp[0][1]['data'] for tmp in sweeps]

In [22]:
step = abs(frequencies[0] - frequencies[-1]) / len(frequencies)
front_padding_count = max(int(frequencies[0] / step), 0)

In [23]:
time_domain_data = []
for i in range(len(sweeps)):
    data_pt = sweeps[i]

    padded_data = []

    while len(padded_data) < front_padding_count:
        padded_data.append(0)
    padded_data.extend(data_pt)

    powers_of_two = [2 ** x for x in range(16)]

    for size in powers_of_two:
        if (size > len(padded_data)):
            final_size = size
            break
    
    while len(padded_data) < final_size:
        padded_data.append(0)
    
    padded_data = np.array(padded_data)

    time_domain_data.append(np.fft.ifft(padded_data))
    
time_domain_data = np.abs(time_domain_data[:len(time_domain_data) // 2])
axis = np.array(range(time_domain_data.shape[1]))

step = step * 1e6 # Hertz
axis = axis * (1 / (len(axis) * step * 2)) # Hertz to seconds
axis = axis * 1e9 # Nanoseconds
axis = axis - CABLE_DELAYS
axis = axis * 0.983571 # Nanoseconds to feet
axis = axis * .5

In [24]:
p = figure(plot_width=1000, plot_height=600, y_axis_type='log', y_range=(10 ** -2, 15e2))
l = p.line(axis, time_domain_data[0], line_width=1)

In [25]:
def update(sweep):
    l.data_source.data['y'] = time_domain_data[sweep - 1]
    push_notebook()

In [30]:
show(p, notebook_handle=True);
interact(update, sweep=(1,1000,1));
interact(pick_capture, capture=DATA_DIRS);

interactive(children=(IntSlider(value=500, description='sweep', max=1000, min=1), Output()), _dom_classes=('wi…

interactive(children=(Dropdown(description='capture', options=('2019-01-31 16-10-08', '2019-01-31 16-01-43', '…